In [ ]:
from math import isqrt
import os.path
from time import perf_counter

import numpy as np
from scipy.special import gamma, gammaln, gammainc
import scipy.linalg as la
import scipy.integrate as spi
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4
import matplotlib.animation as animation

import bin_model as bm

In [ ]:
# Define physical constants.
RHO_WATER = 1000. # Density of water (kg/m^3)
RHO_AIR = 1.2 # Density of air (kg/m^3)

# Model physical parameters.
RAIN_D = 1.e-4 # Cutoff diameter between particle sizes defined as cloud vs. rain (m).

# Grid parameters
D_MIN = 1.e-6 # Minimum particle diameter (m).
D_MAX = 1.e-3 # Maximum particle diameter (m).
NUM_BINS = 90 # Number of evenly spaced bins in the simulation

# Initial conditions
INITIAL_MASS = 8.e-4 # Initial mass concentration (kg/m^3)
INITIAL_NC = 200. # Initial number concentration (cm^-3)
INITIAL_NU = 11. # Shape parameter for initial condition

# Numerical tuning parameters.
STD_DIAMETER = 1.e-4 # Internal scaling for particle size (m)
MASS_CONC_SCALE = INITIAL_MASS
# Long's kernel magnitude kc (m^3/kg^2/s)
long_kernel_size = 9.44e9
TIME_SCALE = 1. / (long_kernel_size * ((np.pi*RHO_WATER/6.)*STD_DIAMETER**3)
                       * MASS_CONC_SCALE)

In [ ]:
const = bm.ModelConstants(rho_water=RHO_WATER, rho_air=RHO_AIR, std_diameter=STD_DIAMETER,
                          rain_d=RAIN_D, mass_conc_scale=MASS_CONC_SCALE,
                          time_scale=TIME_SCALE)
kernel = bm.LongKernel(const)
grid = bm.GeometricMassGrid(const, d_min=D_MIN, d_max=D_MAX, num_bins=NUM_BINS)
ktens = bm.KernelTensor(kernel, grid)

In [ ]:
mass_init = INITIAL_MASS
m3_scaled_init = mass_init / const.std_mass # kg^-1
m3_init = mass_init / (const.rho_water * np.pi/6.) # m^3 / kg 3rd moment
m0_init = INITIAL_NC * 1.e6 * const.rho_air # kg^-1 number concentration
lambda_init = ( m0_init * gamma(INITIAL_NU + 3)/ (m3_init * gamma(INITIAL_NU)) )**(1./3.) # m^-1 scale parameter

In [ ]:
dsd_deriv_names = ['lambda', 'nu']
dsd_deriv_scales = [const.std_diameter, 1.]
desc = bm.ModelStateDescriptor(const, grid,
                               dsd_deriv_names=dsd_deriv_names,
                               dsd_deriv_scales=dsd_deriv_scales)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((2, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
raw = desc.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state = bm.ModelState(desc, raw)

In [ ]:
nb = grid.num_bins
y_init = np.zeros((3*nb,))
y_init[:nb] = desc.dsd_raw(raw)
y_init[nb:2*nb] = desc.dsd_deriv_raw(raw, 'lambda')
y_init[2*nb:] = desc.dsd_deriv_raw(raw, 'nu')

In [ ]:
tscale = const.time_scale
end_time = 5400.
num_time_steps = 120
dt = end_time / num_time_steps
integrator = bm.RK45Integrator(const, dt)
t_eval, states = integrator.integrate(end_time, init_state, [ktens])
all_y = np.zeros((len(t_eval), 3*grid.num_bins,))
for i in range(len(t_eval)):
    raw = states[i].raw
    all_y[i,:grid.num_bins] = desc.dsd_raw(raw)
    all_y[i,grid.num_bins:2*grid.num_bins] = desc.dsd_deriv_raw(raw,
                                                                'lambda')
    all_y[i,2*grid.num_bins:] = desc.dsd_deriv_raw(raw, 'nu')

In [ ]:
bin_midpoints = 0.5 * (grid.bin_bounds[:-1] + grid.bin_bounds[1:])
plot_lxs = 2. + bin_midpoints / (3.*np.log(10))

In [ ]:
mass_convert = mass_init * 1.e3 * (3.*np.log(10))
ymin = 0.
ymax = 0.4*mass_convert
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + grid.lx_min/(3.*np.log(10)), 2. + grid.lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.set_ylabel("$dm/dlog_{10}(D)$ (g/kg)")
ax.grid()

line, = ax.plot(plot_lxs, y_init[:grid.num_bins], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(2., ymin, ymax, 'k')

def animate(i):
    thisy = mass_convert*all_y[i,:grid.num_bins]
    line.set_data(plot_lxs, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(
    fig, animate, len(t_eval), interval=dt*1000*(5./end_time), blit=True)
ani.save("mass_evolution.gif")
plt.show()

In [ ]:
m3_init

In [ ]:
print(init_state.dsd_moment(3))

In [ ]:
final = states[-1].dsd_moment(3)
fallout = states[-1].fallout()/(np.pi*const.rho_water/6.)
print(final+fallout, final, fallout)

In [ ]:
def calc_moment_and_gradient(state, dfdt_val, n, cloud_only=None, rain_only=None):
    weight_vector = grid.moment_weight_vector(n, cloud_only, rain_only)
    mom, grad = state.linear_func_raw(weight_vector, derivative=True, dfdt=dfdt_val)
    scale = const.mass_conc_scale * const.std_diameter**(n) / const.std_mass
    mom *= scale
    grad *= scale
    return mom, grad / mom

In [ ]:
def dfdt_and_deriv(state, proc_tens):
    nb = grid.num_bins
    dsd_raw = state.desc.dsd_raw(state.raw)
    dfdt = np.zeros((nb,))
    ddn = state.desc.dsd_deriv_num
    deriv_tot = np.zeros((nb, nb))
    for pt in proc_tens:
        rate, derivative = pt.calc_rate(dsd_raw, out_flux=True, derivative=True)
        dfdt += rate[:nb]
        deriv_tot += derivative[:nb,:nb]
    dsd_deriv_raw = state.desc.dsd_deriv_raw(state.raw)
    deriv_len = ddn + 1
    dfdt_deriv = np.zeros((deriv_len, nb))
    dfdt_deriv[0,:] += deriv_tot @ dfdt
    dfdt_deriv[1:,:] += dsd_deriv_raw @ deriv_tot.T
    return dfdt, dfdt_deriv

In [ ]:
def moment_rate_and_grad(state, dfdt, dfdt_deriv, n,
                         cloud_only=None, rain_only=None):
    weight_vector = grid.moment_weight_vector(n, cloud_only, rain_only)
    rate, grad = state.linear_func_rate_raw(weight_vector, dfdt,
                                            dfdt_deriv=dfdt_deriv)
    scale = const.mass_conc_scale * const.std_diameter**(n) / const.std_mass \
        / const.time_scale
    rate *= scale
    grad *= scale
    return rate, grad / rate

In [ ]:
mom0s = np.zeros((num_time_steps+1,))
mom3s = np.zeros((num_time_steps+1,))
mom6s = np.zeros((num_time_steps+1,))
mom9s = np.zeros((num_time_steps+1,))
mom_jacobian = np.zeros((num_time_steps+1, 4, 3))
autos = np.zeros((num_time_steps+1,))
accrs = np.zeros((num_time_steps+1,))
cloud_lx = np.log(const.rain_m)
cloud_idx = grid.find_bin(cloud_lx)
if (grid.bin_bounds[cloud_idx+1] - cloud_lx) < 1.e-10:
    cloud_idx += 1
cloud_vector = np.zeros((nb,))
cloud_vector[:cloud_idx] = 1.
for i in range(num_time_steps+1):
    dfdt_val = states[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    mom0s[i], mom_jacobian[i,0,:] = calc_moment_and_gradient(states[i], dfdt_val, 0)
    mom3s[i], mom_jacobian[i,1,:] = calc_moment_and_gradient(states[i], dfdt_val, 3)
    mom6s[i], mom_jacobian[i,2,:] = calc_moment_and_gradient(states[i], dfdt_val, 6)
    mom9s[i], mom_jacobian[i,3,:] = calc_moment_and_gradient(states[i], dfdt_val, 9)
    autos[i], accrs[i] = states[i].rain_prod_breakdown(ktens, cloud_vector)

In [ ]:
momc0s = np.zeros((num_time_steps+1,))
momc3s = np.zeros((num_time_steps+1,))
momr0s = np.zeros((num_time_steps+1,))
momr3s = np.zeros((num_time_steps+1,))
mom_2cat_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    dfdt_val = states[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    momc0s[i], mom_2cat_jacobian[i,0,:] = calc_moment_and_gradient(states[i], dfdt_val, 0,
                                                                   cloud_only=True)
    momc3s[i], mom_2cat_jacobian[i,1,:] = calc_moment_and_gradient(states[i], dfdt_val, 3,
                                                                   cloud_only=True)
    momr0s[i], mom_2cat_jacobian[i,2,:] = calc_moment_and_gradient(states[i], dfdt_val, 0,
                                                                   rain_only=True)
    momr3s[i], mom_2cat_jacobian[i,3,:] = calc_moment_and_gradient(states[i], dfdt_val, 3,
                                                                   rain_only=True)

In [ ]:
mom0s = np.zeros((num_time_steps+1,))
mom3s = np.zeros((num_time_steps+1,))
mom4s = np.zeros((num_time_steps+1,))
mom5s = np.zeros((num_time_steps+1,))
mom_small_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    dfdt_val = states[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    mom0s[i], mom_small_jacobian[i,0,:] = calc_moment_and_gradient(states[i], dfdt_val, 0)
    mom3s[i], mom_small_jacobian[i,1,:] = calc_moment_and_gradient(states[i], dfdt_val, 3)
    mom4s[i], mom_small_jacobian[i,2,:] = calc_moment_and_gradient(states[i], dfdt_val, 4)
    mom5s[i], mom_small_jacobian[i,3,:] = calc_moment_and_gradient(states[i], dfdt_val, 5)

In [ ]:
mom_6_jacobian = np.zeros((num_time_steps+1, 6, 3))
mom_6_jacobian[:,0,:] = mom_small_jacobian[:,0,:]
mom_6_jacobian[:,3:,:] = mom_small_jacobian[:,1:,:]
mom1s = np.zeros((num_time_steps+1,))
mom2s = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    dfdt_val = states[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    mom1s[i], mom_6_jacobian[i,1,:] = calc_moment_and_gradient(states[i], dfdt_val, 1)
    mom2s[i], mom_6_jacobian[i,2,:] = calc_moment_and_gradient(states[i], dfdt_val, 2)

In [ ]:
mom0_rates = np.zeros((num_time_steps+1,))
mom3_rates = np.zeros((num_time_steps+1,))
mom6_rates = np.zeros((num_time_steps+1,))
mom9_rates = np.zeros((num_time_steps+1,))
process_mom_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 0)
    mom0_rates[i], process_mom_jacobian[i,0,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 3)
    mom3_rates[i], process_mom_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 6)
    mom6_rates[i], process_mom_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 9)
    mom9_rates[i], process_mom_jacobian[i,3,:] = rate_out

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, mom9_rates)

In [ ]:
sigma_mom0_rate = np.zeros((num_time_steps+1,))
sigma_mom3_rate = np.zeros((num_time_steps+1,))
sigma_mom6_rate = np.zeros((num_time_steps+1,))
sigma_mom9_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_jacobian[i,:,:])
    pj = process_mom_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_jacobian[i,0,:])
    pj[1,:] = mom3_rates[i] / mom3s[i] * (pj[1,:] - mom_jacobian[i,1,:])
    pj[2,:] = mom6_rates[i] / mom6s[i] * (pj[2,:] - mom_jacobian[i,2,:])
    pj[3,:] = mom9_rates[i] / mom9s[i] * (pj[3,:] - mom_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_mom3_rate[i] = np.sqrt(cov[1,1])
    sigma_mom6_rate[i] = np.sqrt(cov[2,2])
    sigma_mom9_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, sigma_mom0_rate, label="M0")
#plt.plot(t_eval, sigma_mom3_rate)
plt.plot(t_eval, sigma_mom6_rate, label="M6")
plt.plot(t_eval, sigma_mom9_rate, label="M9")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m0369_predictability.png')

In [ ]:
mom4_rates = np.zeros((num_time_steps+1,))
mom5_rates = np.zeros((num_time_steps+1,))
process_small_jacobian = np.zeros((num_time_steps+1, 4, 3))
process_small_jacobian[:,:2,:] = process_mom_jacobian[:,:2,:]
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 4)
    mom4_rates[i], process_small_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 5)
    mom5_rates[i], process_small_jacobian[i,3,:] = rate_out

In [ ]:
sigma_small_mom0_rate = np.zeros((num_time_steps+1,))
sigma_small_mom3_rate = np.zeros((num_time_steps+1,))
sigma_small_mom4_rate = np.zeros((num_time_steps+1,))
sigma_small_mom5_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_small_jacobian[i,:,:])
    pj = process_small_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_small_jacobian[i,0,:])
    pj[1,:] = mom3_rates[i] / mom3s[i] * (pj[1,:] - mom_small_jacobian[i,1,:])
    pj[2,:] = mom4_rates[i] / mom4s[i] * (pj[2,:] - mom_small_jacobian[i,2,:])
    pj[3,:] = mom5_rates[i] / mom5s[i] * (pj[3,:] - mom_small_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_small_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_small_mom3_rate[i] = np.sqrt(cov[1,1])
    sigma_small_mom4_rate[i] = np.sqrt(cov[2,2])
    sigma_small_mom5_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, sigma_small_mom0_rate, label="M0")
#plt.plot(t_eval, sigma_small_mom3_rate)
plt.plot(t_eval, sigma_small_mom4_rate, label="M4")
plt.plot(t_eval, sigma_small_mom5_rate, label="M5")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m0345_predictability.png')

In [ ]:
fig = plt.figure(figsize=(5, 4))
ax = fig.gca()
ax.plot(t_eval, 10.*np.log10(mom5s), 'k')
ax.plot(t_eval, 10.*np.log10(mom5_rates), 'r--')

In [ ]:
fig = plt.figure(figsize=(5, 4))
ax = fig.gca()
ax.plot(t_eval, 10.*np.log10(mom9s), 'k')
ax.plot(t_eval, 10.*np.log10(mom9_rates), 'r--')

In [ ]:
momc0_rates = np.zeros((num_time_steps+1,))
momc3_rates = np.zeros((num_time_steps+1,))
momr0_rates = np.zeros((num_time_steps+1,))
momr3_rates = np.zeros((num_time_steps+1,))
process_2cat_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 0,
                                    cloud_only=True)
    momc0_rates[i], process_2cat_jacobian[i,0,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 3,
                                    cloud_only=True)
    momc3_rates[i], process_2cat_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 0,
                                    rain_only=True)
    momr0_rates[i], process_2cat_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 3,
                                    rain_only=True)
    momr3_rates[i], process_2cat_jacobian[i,3,:] = rate_out

In [ ]:
sigma_2cat_momc0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momc3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momr0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momr3_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_2cat_jacobian[i,:,:])
    pj = process_2cat_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_jacobian[i,1,:])
    pj[2,:] = momr0_rates[i] / momr0s[i] * (pj[2,:] - mom_2cat_jacobian[i,2,:])
    pj[3,:] = momr3_rates[i] / momr3s[i] * (pj[3,:] - mom_2cat_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_2cat_momc0_rate[i] = np.sqrt(cov[0,0])
    sigma_2cat_momc3_rate[i] = np.sqrt(cov[1,1])
    sigma_2cat_momr0_rate[i] = np.sqrt(cov[2,2])
    sigma_2cat_momr3_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, sigma_2cat_momc0_rate, label="Cloud M0")
plt.plot(t_eval, sigma_2cat_momc3_rate, label="Cloud M3")
plt.plot(t_eval, sigma_2cat_momr0_rate, label="Rain M0")
plt.plot(t_eval, sigma_2cat_momr3_rate, label="Rain M3")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('cm03_rm03_predictability.png')

In [ ]:
process_6_jacobian = np.zeros((num_time_steps+1, 6, 3))
process_6_jacobian[:,0,:] = process_small_jacobian[:,0,:]
process_6_jacobian[:,3:,:] = process_small_jacobian[:,1:,:]
mom1_rates = np.zeros((num_time_steps+1,))
mom2_rates = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 1)
    mom1_rates[i], process_small_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 2)
    mom2_rates[i], process_small_jacobian[i,2,:] = rate_out

In [ ]:
sigma_6_mom0_rate = np.zeros((num_time_steps+1,))
sigma_6_mom1_rate = np.zeros((num_time_steps+1,))
sigma_6_mom2_rate = np.zeros((num_time_steps+1,))
sigma_6_mom3_rate = np.zeros((num_time_steps+1,))
sigma_6_mom4_rate = np.zeros((num_time_steps+1,))
sigma_6_mom5_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_6_jacobian[i,:,:])
    pj = process_6_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_6_jacobian[i,0,:])
    pj[1,:] = mom1_rates[i] / mom1s[i] * (pj[1,:] - mom_6_jacobian[i,1,:])
    pj[2,:] = mom2_rates[i] / mom2s[i] * (pj[2,:] - mom_6_jacobian[i,2,:])
    pj[3,:] = mom3_rates[i] / mom3s[i] * (pj[3,:] - mom_6_jacobian[i,3,:])
    pj[4,:] = mom4_rates[i] / mom4s[i] * (pj[4,:] - mom_6_jacobian[i,4,:])
    pj[5,:] = mom5_rates[i] / mom5s[i] * (pj[5,:] - mom_6_jacobian[i,5,:])
    cov = pj @ mom_inv @ np.eye(6) @ mom_inv.T @ pj.T
    sigma_6_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_6_mom1_rate[i] = np.sqrt(cov[1,1])
    sigma_6_mom2_rate[i] = np.sqrt(cov[2,2])
    sigma_6_mom3_rate[i] = np.sqrt(cov[3,3])
    sigma_6_mom4_rate[i] = np.sqrt(cov[4,4])
    sigma_6_mom5_rate[i] = np.sqrt(cov[5,5])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, sigma_6_mom0_rate, label="M0")
plt.plot(t_eval, sigma_6_mom1_rate, label="M1")
plt.plot(t_eval, sigma_6_mom2_rate, label="M2")
plt.plot(t_eval, sigma_6_mom4_rate, label="M4")
plt.plot(t_eval, sigma_6_mom5_rate, label="M5")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m012345_predictability.png')

In [ ]:
momc6s = np.zeros((num_time_steps+1,))
momr6s = np.zeros((num_time_steps+1,))
mom_2cat_3m_jacobian = np.zeros((num_time_steps+1, 6, 3))
mom_2cat_3m_jacobian[:,:2,:] = mom_2cat_jacobian[:,:2,:]
mom_2cat_3m_jacobian[:,3:5,:] = mom_2cat_jacobian[:,2:,:]
for i in range(num_time_steps+1):
    dfdt_val = states[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    mom_out = calc_moment_and_gradient(states[i], dfdt_val, 6, cloud_only=True)
    momc6s[i], mom_2cat_3m_jacobian[i,2,:] = mom_out
    mom_out = calc_moment_and_gradient(states[i], dfdt_val, 6, rain_only=True)
    momr6s[i], mom_2cat_3m_jacobian[i,5,:] = mom_out

In [ ]:
momc6_rates = np.zeros((num_time_steps+1,))
momr6_rates = np.zeros((num_time_steps+1,))
process_2cat_3m_jacobian = np.zeros((num_time_steps+1, 6, 3))
process_2cat_3m_jacobian[:,:2,:] = process_2cat_jacobian[:,:2,:]
process_2cat_3m_jacobian[:,3:5,:] = process_2cat_jacobian[:,2:,:]
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 6,
                                    cloud_only=True)
    momc6_rates[i], process_2cat_3m_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, 6,
                                    rain_only=True)
    momr6_rates[i], process_2cat_3m_jacobian[i,5,:] = rate_out

In [ ]:
sigma_2cat_3m_momc0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momc3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momc6_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr6_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_2cat_3m_jacobian[i,:,:])
    pj = process_2cat_3m_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_3m_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_3m_jacobian[i,1,:])
    pj[2,:] = momc6_rates[i] / momc6s[i] * (pj[2,:] - mom_2cat_3m_jacobian[i,2,:])
    pj[3,:] = momr0_rates[i] / momr0s[i] * (pj[3,:] - mom_2cat_3m_jacobian[i,3,:])
    pj[4,:] = momr3_rates[i] / momr3s[i] * (pj[4,:] - mom_2cat_3m_jacobian[i,4,:])
    pj[5,:] = momr6_rates[i] / momr6s[i] * (pj[5,:] - mom_2cat_3m_jacobian[i,5,:])
    cov = pj @ mom_inv @ np.eye(6) @ mom_inv.T @ pj.T
    sigma_2cat_3m_momc0_rate[i] = np.sqrt(cov[0,0])
    sigma_2cat_3m_momc3_rate[i] = np.sqrt(cov[1,1])
    sigma_2cat_3m_momc6_rate[i] = np.sqrt(cov[2,2])
    sigma_2cat_3m_momr0_rate[i] = np.sqrt(cov[3,3])
    sigma_2cat_3m_momr3_rate[i] = np.sqrt(cov[4,4])
    sigma_2cat_3m_momr6_rate[i] = np.sqrt(cov[5,5])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, sigma_2cat_3m_momc0_rate, label="Cloud M0")
#plt.plot(t_eval, sigma_2cat_momc3_rate)
plt.plot(t_eval, sigma_2cat_3m_momc6_rate, label="Cloud M6")
plt.plot(t_eval, sigma_2cat_3m_momr0_rate, label="Rain M0")
plt.plot(t_eval, sigma_2cat_3m_momr3_rate, label="Rain M3")
plt.plot(t_eval, sigma_2cat_3m_momr6_rate, label="Rain M6")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('cm036_rm036_predictability.png')

# Comparison plots

In [ ]:
for i in range(num_time_steps+1):
    if momr3s[i] >= 0.99*mom3s[i]:
        half_time = i//2
        print(t_eval[half_time])
        break

In [ ]:
nmom = 10
full_moms = np.zeros((num_time_steps+1, nmom))
full_mom_rates = np.zeros((num_time_steps+1, nmom))
full_mom_jacobian = np.zeros((num_time_steps+1, nmom, 3))
full_mom_rate_jacobian = np.zeros((num_time_steps+1, nmom, 3))
for i in range(num_time_steps+1):
    dfdt_val, dfdt_deriv = dfdt_and_deriv(states[i], [ktens])
    for n in range(nmom):
        mom_out = calc_moment_and_gradient(states[i], dfdt_val, n)
        full_moms[i,n], full_mom_jacobian[i,n,:] = mom_out
        rate_out = moment_rate_and_grad(states[i], dfdt_val, dfdt_deriv, n)
        full_mom_rates[i,n], full_mom_rate_jacobian[i,n,:] = rate_out

In [ ]:
full_scaled_rate_jacobian = np.zeros((num_time_steps+1, nmom, 3))
for i in range(num_time_steps+1):
    mj = full_mom_jacobian[i,:,:]
    pj = full_mom_rate_jacobian[i,:,:]
    for n in range(nmom):
        full_scaled_rate_jacobian[i,n,:] = full_mom_rates[i,n] / full_moms[i,n] * (pj[n,:] - mj[n,:])

In [ ]:
moments = [0, 4, 5]
nmom = len(moments)
j_eigs = np.zeros((num_time_steps+1, nmom), dtype=np.complex128)
j_evals = np.zeros((num_time_steps+1, nmom, nmom), dtype=np.complex128)
for i in range(num_time_steps):
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    j_eigs[i,:], j_evals[i,:,:] = la.eig(pj @ mom_inv)

In [ ]:
print(j_eigs[0,:], j_evals[0,:,:])

In [ ]:
plt.scatter(np.repeat(t_eval, len(moments)), np.ravel(np.real(j_eigs)))

In [ ]:
plt.scatter(np.repeat(t_eval, len(moments)), np.ravel(np.imag(j_eigs)))

In [ ]:
full_mom_jacobian[0,[0,3,6,9],:]

In [ ]:
mom_jacobian[0,:,:]

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps):
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    cov = pj @ mom_inv @ np.eye(nmom) @ mom_inv.T @ pj.T
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
initial_error = 0.#0005 * np.log(10.)
error_per_sec = 0.000005 * np.log(10.) / 60.

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps):
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    cov = pj @ mom_inv @ np.eye(nmom)# @ mom_inv.T @ pj.T
    cov = cov + cov.T
    for n in range(nmom):
        sigma_diags[i,n] = cov[n,n]#np.sqrt(cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, -ymax, ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, -ymax, ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([-ymax, ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
nmom = 4
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps):
    mom_inv = la.pinv(mom_2cat_jacobian[i,:,:])
    pj = process_2cat_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_jacobian[i,1,:])
    pj[2,:] = momr0_rates[i] / momr0s[i] * (pj[2,:] - mom_2cat_jacobian[i,2,:])
    pj[3,:] = momr3_rates[i] / momr3s[i] * (pj[3,:] - mom_2cat_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(nmom)# @ mom_inv.T @ pj.T
    cov = cov + cov.T
    for n in range(nmom):
        sigma_diags[i,n] = cov[n,n]#np.sqrt(cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n])
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, -ymax, ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, -ymax, ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([-ymax, ymax])
plt.legend(loc='best')

In [ ]:
nmom = 4
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(mom_2cat_jacobian[i,:,:])
    pj = process_2cat_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_jacobian[i,1,:])
    pj[2,:] = momr0_rates[i] / momr0s[i] * (pj[2,:] - mom_2cat_jacobian[i,2,:])
    pj[3,:] = momr3_rates[i] / momr3s[i] * (pj[3,:] - mom_2cat_jacobian[i,3,:])
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n])
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
nmom = 6
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    proj = mom_2cat_3m_jacobian[i,:,:]
    inv_part = la.inv(proj.T @ np.eye(nmom) @ proj)
    proj_full = proj @ inv_part @ proj.T @ la.inv(np.eye(nmom))
    cum_cov = proj_full @ cum_cov @ proj_full.T
    mom_inv = la.pinv(mom_2cat_3m_jacobian[i,:,:])
    pj = process_2cat_3m_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_3m_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_3m_jacobian[i,1,:])
    pj[2,:] = momr0_rates[i] / momr0s[i] * (pj[2,:] - mom_2cat_3m_jacobian[i,2,:])
    pj[3,:] = momr3_rates[i] / momr3s[i] * (pj[3,:] - mom_2cat_3m_jacobian[i,3,:])
    pj[4,:] = momr3_rates[i] / momr3s[i] * (pj[4,:] - mom_2cat_3m_jacobian[i,4,:])
    pj[5,:] = momr6_rates[i] / momr6s[i] * (pj[5,:] - mom_2cat_3m_jacobian[i,5,:])
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Cloud M6", "Rain M0", "Rain M3", "Rain M6"]
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n])
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 1, 2]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 4, 5]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 5, 6]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wv0 = grid.moment_weight_vector(0)
wv3 = grid.moment_weight_vector(3)
wv6 = grid.moment_weight_vector(6)
wv9 = grid.moment_weight_vector(9)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wv0, bm.LogTransform(), db_scale),
    (wv3, bm.LogTransform(), db_scale),
    (wv6, bm.LogTransform(), db_scale),
    (wv9, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one hour
perturbation_rate = error_rate**2 * np.eye(4) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
desc2 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw2 = desc2.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state2 = bm.ModelState(desc2, raw2)

In [ ]:
t_eval2, states2 = integrator.integrate(end_time, init_state2, [ktens])

In [ ]:
moments = [0, 3, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states2[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wvc0 = grid.moment_weight_vector(0, cloud_only=True)
wvc3 = grid.moment_weight_vector(3, cloud_only=True)
wvr0 = grid.moment_weight_vector(0, rain_only=True)
wvr3 = grid.moment_weight_vector(3, rain_only=True)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wvc0, bm.LogTransform(), db_scale),
    (wvc3, bm.LogTransform(), db_scale),
    (wvr0, bm.LogTransform(), db_scale),
    (wvr3, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one day
perturbation_rate = error_rate**2 * np.eye(4) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
desc3 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw3 = desc3.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state3 = bm.ModelState(desc3, raw3)

In [ ]:
t_eval3, states3 = integrator.integrate(end_time, init_state3, [ktens])

In [ ]:
nmom = 4
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states3[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Rain M0", "Rain M3"]
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n])
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
wvc0 = grid.moment_weight_vector(0, cloud_only=True)
wvc3 = grid.moment_weight_vector(3, cloud_only=True)
wvc6 = grid.moment_weight_vector(6, cloud_only=True)
wvr0 = grid.moment_weight_vector(0, rain_only=True)
wvr3 = grid.moment_weight_vector(3, rain_only=True)
wvr6 = grid.moment_weight_vector(6, rain_only=True)
db_scale = 10. / np.log(10.)
perturbed_variables = [
    (wvc0, bm.LogTransform(), db_scale),
    (wvc3, bm.LogTransform(), db_scale),
    (wvc6, bm.LogTransform(), db_scale),
    (wvr0, bm.LogTransform(), db_scale),
    (wvr3, bm.LogTransform(), db_scale),
    (wvr6, bm.LogTransform(), db_scale),
]
error_rate = 1. # db after one day
perturbation_rate = error_rate**2 * np.eye(6) / 3600.
dsd_deriv_names = ["lambda", "nu", "M3"]
dsd_deriv_scales = [const.std_diameter, 1., 1 / const.mass_conc_scale]
correction_time = 5.
desc4 = bm.ModelStateDescriptor(const, grid,
                                dsd_deriv_names=dsd_deriv_names,
                                dsd_deriv_scales=dsd_deriv_scales,
                                perturbed_variables=perturbed_variables,
                                perturbation_rate=perturbation_rate,
                                correction_time=correction_time)
dsd = bm.gamma_dist_d(grid, lambda_init, INITIAL_NU)
dsd_scale = mass_init / np.dot(dsd, grid.bin_widths)
dsd *= dsd_scale
dsd_deriv = np.zeros((3, grid.num_bins))
dsd_deriv[0,:] = bm.gamma_dist_d_lam_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[1,:] = bm.gamma_dist_d_nu_deriv(grid, lambda_init, INITIAL_NU) * dsd_scale
dsd_deriv[2,:] = dsd / const.mass_conc_scale
raw4 = desc4.construct_raw(dsd, dsd_deriv=dsd_deriv)
init_state4 = bm.ModelState(desc4, raw4)

In [ ]:
t_eval4, states4 = integrator.integrate(end_time, init_state4, [ktens])

In [ ]:
nmom = 6
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps+1):
    perturb_cov = states4[i].perturb_cov()
    for j in range(nmom):
        sigma_diags[i,j] = np.sqrt(perturb_cov[j,j])

In [ ]:
fig = plt.figure(figsize=(5, 4))
labels = ["Cloud M0", "Cloud M3", "Cloud M6", "Rain M0", "Rain M3", "Rain M6"]
colors = ['tab:blue', 'tab:orange', 'tab:purple', 'tab:green', 'tab:red', 'tab:brown']
for n in range(nmom):
    plt.plot(t_eval, sigma_diags[:,n], label=labels[n], color=colors[n])
ymax = 10.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
covar_volume_single_cat = np.zeros((num_time_steps+1,))
covar_volume_two_cat_2m = np.zeros((num_time_steps+1,))
covar_volume_two_cat_3m = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    perturb_cov = states2[i].perturb_cov()
    covar_volume_single_cat[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = states3[i].perturb_cov()
    covar_volume_two_cat_2m[i] = np.prod(la.svdvals(perturb_cov)[:4])
    perturb_cov = states4[i].perturb_cov()
    covar_volume_two_cat_3m[i] = np.prod(la.svdvals(perturb_cov)[:4])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, covar_volume_single_cat, label='1-cat 0369')
plt.plot(t_eval, covar_volume_two_cat_2m, label='2-cat c03r03')
plt.plot(t_eval, covar_volume_two_cat_3m, label='2-cat c036r036')
#ymax = 4.5e-6
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
ddsddt_single_cat = np.zeros((num_time_steps+1, grid.num_bins))
ddsddt_two_cat_2m = np.zeros((num_time_steps+1, grid.num_bins))
ddsddt_two_cat_3m = np.zeros((num_time_steps+1, grid.num_bins))
zeta_covar_single_cat = np.zeros((num_time_steps+1, 4, 4))
zeta_covar_two_cat_2m = np.zeros((num_time_steps+1, 4, 4))
zeta_covar_two_cat_3m = np.zeros((num_time_steps+1, 4, 4))
for i in range(num_time_steps+1):
    ddsddt_single_cat[i,:] = states2[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    lfs = np.zeros((4,))
    lf_jac = np.zeros((4, 4))
    for j in range(4):
        wv = desc2.perturb_wvs[j]
        lfs[j], lf_jac[j,:] = states2[i].linear_func_raw(wv, derivative=True,
                                                         dfdt=ddsddt_single_cat[i,:])
    transform_mat = np.diag([desc2.perturb_transforms[j].derivative(lfs[j])
                             for j in range(4)])
    v_to_zeta = la.pinv(transform_mat @ lf_jac)
    perturb_cov = desc2.perturb_cov_raw(states2[i].raw)
    zeta_covar_single_cat[i,:,:] = v_to_zeta @ perturb_cov @ v_to_zeta.T
    ddsddt_two_cat_2m[i,:] = states3[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    lfs = np.zeros((4,))
    lf_jac = np.zeros((4, 4))
    for j in range(4):
        wv = desc3.perturb_wvs[j]
        lfs[j], lf_jac[j,:] = states3[i].linear_func_raw(wv, derivative=True,
                                                         dfdt=ddsddt_two_cat_2m[i,:])
    transform_mat = np.diag([desc3.perturb_transforms[j].derivative(lfs[j])
                             for j in range(4)])
    v_to_zeta = la.pinv(transform_mat @ lf_jac)
    perturb_cov = desc3.perturb_cov_raw(states3[i].raw)
    zeta_covar_two_cat_2m[i,:,:] = v_to_zeta @ perturb_cov @ v_to_zeta.T
    ddsddt_two_cat_3m[i,:] = states4[i].dsd_time_deriv_raw([ktens])[:grid.num_bins]
    lfs = np.zeros((6,))
    lf_jac = np.zeros((6, 4))
    for j in range(6):
        wv = desc4.perturb_wvs[j]
        lfs[j], lf_jac[j,:] = states4[i].linear_func_raw(wv, derivative=True,
                                                         dfdt=ddsddt_two_cat_3m[i,:])
    transform_mat = np.diag([desc4.perturb_transforms[j].derivative(lfs[j])
                             for j in range(6)])
    v_to_zeta = la.pinv(transform_mat @ lf_jac)
    perturb_cov = desc4.perturb_cov_raw(states4[i].raw)
    zeta_covar_two_cat_3m[i,:,:] = v_to_zeta @ perturb_cov @ v_to_zeta.T

In [ ]:
cm3_amount = np.zeros((num_time_steps+1,))
single_cat_cm3_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_cm3_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_cm3_uncertainty = np.zeros((num_time_steps+1,))
cm3_vector = grid.moment_weight_vector(3, cloud_only=True)
for i in range(num_time_steps+1):
    cm3_amount[i], deriv = states2[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_cm3_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    _, deriv = states3[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_cm3_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    _, deriv = states4[i].linear_func_raw(cm3_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_cm3_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
scale = const.mass_conc_scale / mass_init
single_cat_cm3_uncertainty *= scale
two_cat_2m_cm3_uncertainty *= scale
two_cat_3m_cm3_uncertainty *= scale
cm3_amount *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, cm3_amount, 'k', label='Total remaining')
plt.plot(t_eval, single_cat_cm3_uncertainty, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_cm3_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_cm3_uncertainty, label='2-cat c036r036')
#ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
single_cat_m6 = np.zeros((num_time_steps+1,))
single_cat_m6_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_m6 = np.zeros((num_time_steps+1,))
two_cat_2m_m6_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_m6 = np.zeros((num_time_steps+1,))
two_cat_3m_m6_uncertainty = np.zeros((num_time_steps+1,))
m6_vector = grid.moment_weight_vector(6)
for i in range(num_time_steps+1):
    single_cat_m6[i], deriv = states2[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_m6_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    two_cat_2m_m6[i], deriv = states3[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_m6_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    two_cat_3m_m6[i], deriv = states4[i].linear_func_raw(m6_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_m6_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
scale = const.mass_conc_scale * const.std_diameter**6 / const.std_mass
single_cat_m6_uncertainty = np.sqrt(single_cat_m6_uncertainty) * scale
two_cat_2m_m6_uncertainty = np.sqrt(two_cat_2m_m6_uncertainty) * scale
two_cat_3m_m6_uncertainty = np.sqrt(two_cat_3m_m6_uncertainty) * scale
single_cat_m6 *= scale
two_cat_2m_m6 *= scale
two_cat_3m_m6 *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m6, label='Total M6', color='k')
plt.plot(t_eval, single_cat_m6_uncertainty, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m6_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m6_uncertainty, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m6, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m6, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m6, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
single_cat_m0 = np.zeros((num_time_steps+1,))
single_cat_m0_uncertainty = np.zeros((num_time_steps+1,))
two_cat_2m_m0 = np.zeros((num_time_steps+1,))
two_cat_2m_m0_uncertainty = np.zeros((num_time_steps+1,))
two_cat_3m_m0 = np.zeros((num_time_steps+1,))
two_cat_3m_m0_uncertainty = np.zeros((num_time_steps+1,))
m0_vector = grid.moment_weight_vector(0)
for i in range(num_time_steps+1):
    single_cat_m0[i], deriv = states2[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_single_cat[i,:])
    single_cat_m0_uncertainty[i] = deriv @ zeta_covar_single_cat[i,:,:] @ deriv
    two_cat_2m_m0[i], deriv = states3[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_2m[i,:])
    two_cat_2m_m0_uncertainty[i] = deriv @ zeta_covar_two_cat_2m[i,:,:] @ deriv
    two_cat_3m_m0[i], deriv = states4[i].linear_func_raw(m0_vector, derivative=True,
                                          dfdt=ddsddt_two_cat_3m[i,:])
    two_cat_3m_m0_uncertainty[i] = deriv @ zeta_covar_two_cat_3m[i,:,:] @ deriv
scale = const.mass_conc_scale / const.std_mass
single_cat_m0_uncertainty = np.sqrt(single_cat_m0_uncertainty) * scale
two_cat_2m_m0_uncertainty = np.sqrt(two_cat_2m_m0_uncertainty) * scale
two_cat_3m_m0_uncertainty = np.sqrt(two_cat_3m_m0_uncertainty) * scale
single_cat_m0 *= scale
two_cat_2m_m0 *= scale
two_cat_3m_m0 *= scale

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m0, label='Total M0', color='k')
plt.plot(t_eval, single_cat_m0_uncertainty, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m0_uncertainty, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m0_uncertainty, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval, single_cat_m0, label='1-cat 0369')
plt.plot(t_eval, two_cat_2m_m0, label='2-cat c03r03')
plt.plot(t_eval, two_cat_3m_m0, label='2-cat c036r036')
#ymax = 1.e-17
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i]
        break
#plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
#plt.ylim([0., ymax])
plt.legend(loc='best')